In [11]:
import pandas as pd

df = pd.read_csv("../datasets/encoded/encoded-strutture_sanitare_private.csv", delimiter=';');
df.head()

,CODICEIDENTIFICATIVOFARMACIA,CODFARMACIAASSEGNATODAASL,INDIRIZZO,DESCRIZIONEFARMACIA,PARTITAIVA,CAP,CODICECOMUNEISTAT,DESCRIZIONECOMUNE,FRAZIONE,CODICEPROVINCIAISTAT,...,DESCRIZIONEPROVINCIA,CODICEREGIONE,DESCRIZIONEREGIONE,DATAINIZIOVALIDITA,DATAFINEVALIDITA,DESCRIZIONETIPOLOGIA,CODICETIPOLOGIA,LATITUDINE,LONGITUDINE,LOCALIZE
0,976,0,"Via EUGENIO L'EMIRO, 34/36",VERGA SRL,6654320826,90135,82053,PALERMO,-,82,...,PALERMO,190,SICILIA,45047,-,Ordinaria,1,"45,75803406","8,376775811",1
1,14429,0,"Via Empedocle, 71",AVERNA ANTONIO,141720847,92100,84001,AGRIGENTO,-,84,...,AGRIGENTO,190,SICILIA,40452,43281,Ordinaria,1,"37,31095839","13,58128479",1
2,14430,0,"Via Leonardo Sciascia, 28",CAMILLERI DELLE DR.SSE ANNACAMILLERI E DANIELA...,2366000848,92100,84001,AGRIGENTO,-,84,...,AGRIGENTO,190,SICILIA,41334,43465,Ordinaria,1,"37,27368636","13,61799293",1
3,14431,0,"Via Alessandro Manzoni, 28",NUARA MARCELLA,1528440843,92100,84001,AGRIGENTO,-,84,...,AGRIGENTO,190,SICILIA,38353,43465,Ordinaria,1,"37,3081491","13,5802265",1
4,14432,0,"Via Gioeni, 159",ROMANO GIUSEPPINA,1800710848,92100,84001,AGRIGENTO,-,84,...,AGRIGENTO,190,SICILIA,41334,43465,Ordinaria,1,"37,31437184","13,58147119",1
